In [2]:
import cv2
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
#Matplot Images
import matplotlib.image as mpimg
# Tensflor and Keras Layer and Model and Optimize and Loss
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import *
from tensorflow.keras.losses import BinaryCrossentropy
#PreTrained Model
from tensorflow.keras.applications import *
#Image Generator DataAugmentation
#Early Stopping
from tensorflow.keras.callbacks import EarlyStopping
# Warnings Remove 
import warnings 
warnings.filterwarnings("ignore")
#Splitting Data 
# import splitfolders
from tensorflow.keras.preprocessing import image_dataset_from_directory
import random
from sklearn.cluster import KMeans
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from octave_resnet import *

In [3]:

directory = "C:\\Users\\bvcha\\Desktop\\Senior Design Project\\Datasets\\PaddyDoctor\\Paddy_doctor\\train_images_paddy"

filepath =[]
label = []

folds = os.listdir(directory)

for fold in folds:
    f_path = os.path.join(directory , fold)
    
    imgs = os.listdir(f_path)
    
    for img in imgs:
        
        img_path = os.path.join(f_path , img)
        filepath.append(img_path)
        label.append(fold)
        
#Concat data paths with labels
file_path_series = pd.Series(filepath , name= 'filepath')
Label_path_series = pd.Series(label , name = 'label')
df_train = pd.concat([file_path_series ,Label_path_series ] , axis = 1)

In [4]:
from sklearn. model_selection import train_test_split
train_df , val_df = train_test_split(df_train ,train_size = 0.9 , shuffle = True ,random_state = 42 )

In [5]:
print(f"The shape of The Train data is: {train_df.shape}")
print(f"The shape of The Validation data is: {val_df.shape}")

The shape of The Train data is: (9366, 2)
The shape of The Validation data is: (1041, 2)


In [6]:
data_dir = 'C:\\Users\\bvcha\\Desktop\\Senior Design Project\\Datasets\\PaddyDoctor\\Paddy_doctor\\train_images_paddy'

print('Training Images:')
# creating the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.4,
    subset='training',
    seed=123,
    image_size=(256, 256),
    batch_size=128)

#Testing Augmented Data
print('Validation Images:')
validation_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir, 
    validation_split=0.4,
    subset='validation',
    seed=123,
    image_size=(256, 256),
    batch_size=128)

Training Images:
Found 10407 files belonging to 10 classes.
Using 6245 files for training.
Validation Images:
Found 10407 files belonging to 10 classes.
Using 4162 files for validation.


In [7]:
test_ds = keras.utils.image_dataset_from_directory(
    directory = 'C:\\Users\\bvcha\\Desktop\\Senior Design Project\\Datasets\\PaddyDoctor\\Paddy_doctor\\test_images_paddy',
    batch_size = 128,
    image_size = (256, 256),
    label_mode = None,
    shuffle=False
)
test_ds = validation_ds.map(lambda x, y: (x / 255.0, y))

Found 3469 files belonging to 1 classes.


In [8]:
for image_batch, labels_batch in train_ds:
    print("Shape of X_train: ", image_batch.shape)
    print("Shape of y_train: ", labels_batch.shape)
    break

Shape of X_train:  (128, 256, 256, 3)
Shape of y_train:  (128,)


In [9]:
train_ds = train_ds.map(lambda x, y: (x / 255.0, y))
# Val Data
validation_ds = validation_ds.map(lambda x, y: (x / 255.0, y))

In [10]:
from octave_resnet import OctaveResNet50
model = OctaveResNet50(input_shape=(256, 256, 3), classes=10,
                           alpha=0.125, expansion=4,
                           initial_filters=64,
                           initial_strides=True)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()




Model: "OctaveResNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 64)         9408      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 128, 128, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 re_lu (ReLU)                (None, 128, 128, 64)         0         ['batch_normaliz

In [11]:
early_stopping = EarlyStopping(
    patience=10, 
    restore_best_weights=True,
)

# Train the model 
history = model.fit(train_ds,
                    epochs=5,
                    validation_data=validation_ds,
                    callbacks=early_stopping)

Epoch 1/5


